<a href="https://colab.research.google.com/github/TuyetMaiLe/COVID19-Fake-News-Detection-in-English/blob/main/Stem%2BStop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
!pip install emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import TweetTokenizer 
from emoji import demojize
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer

In [ ]:
from sklearn.model_selection import GridSearchCV 

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
Dataset_train=pd.read_excel("/content/Constraint_English_Train.xlsx")
Dataset_test= pd.read_excel("/content/Constraint_English_Val.xlsx")
Data_train = Dataset_train.drop('id', 1)
Data_test = Dataset_test.drop('id', 1)
Data_test_raw = Dataset_test.drop('id', 1)

In [ ]:
Data_train_1=Dataset_train.drop('id', 1)

In [ ]:
Data_test_1=Dataset_test.drop('id', 1)

In [ ]:
tokenizer = TweetTokenizer()

In [ ]:
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

In [ ]:
def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [ ]:
Data_train.loc[:,"tweet"] = Data_train['tweet'].apply(lambda x: normalizeTweet(x))
Data_train.loc[:,"label"] = Data_train['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_train.columns = ["tweet", "labels"]

In [ ]:
Data_train_1.loc[:,"tweet"] = Data_train_1['tweet'].apply(lambda x: normalizeTweet(x))
Data_train_1.loc[:,"label"] = Data_train_1['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_train_1.columns = ["tweet", "labels"]

In [ ]:
#test đã tiền xử lý
Data_test.loc[:,"tweet"] = Data_test['tweet'].apply(lambda x: normalizeTweet(x))
Data_test.loc[:,"label"] = Data_test['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_test.columns = ["tweet", "labels"]

In [ ]:
Data_test_1.loc[:,"tweet"] = Data_test_1['tweet'].apply(lambda x: normalizeTweet(x))
Data_test_1.loc[:,"label"] = Data_test_1['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_test_1.columns = ["tweet", "labels"]

In [ ]:
Data_train = Data_train.apply(lambda X_train: X_train.astype(str).str.lower())
Data_test = Data_test.apply(lambda X_train: X_train.astype(str).str.lower())

In [ ]:
Data_train

,tweet,labels
0,the cdc currently reports 99031 deaths . in ge...,0
1,states reported 1121 deaths a small rise from ...,0
2,politically correct woman ( almost ) uses pand...,1
3,#indiafightscorona : we have 1524 #covid testi...,0
4,populous states can generate large case counts...,0
...,...,...
6415,a tiger tested positive for covid - 19 please ...,1
6416,? ? ? autopsies prove that covid - 19 is ? ? �...,1
6417,_a post claims a covid - 19 vaccine has alread...,1
6418,aamir khan donate 250 cr . in pm relief cares ...,1



** Train dữ liệu** + **stemword** + **stopword**


In [ ]:
STOPWORDS = set(stopwords.words('english'))
stemmer = PorterStemmer()
def pre_Process_SS(text):
  text= " ".join([word for word in str(text).split() if word not in STOPWORDS])
  text=" ".join([stemmer.stem(word) for word in text.split()])
  return text

In [ ]:
Data_train_1 = Data_train_1.apply(lambda X_train: X_train.astype(str).str.lower())
Data_test_1 = Data_test_1.apply(lambda X_train: X_train.astype(str).str.lower())

In [ ]:
Data_test_1.loc[:,"tweet"] = Data_test_1["tweet"].apply(lambda X_train: pre_Process_SS(X_train))
Data_train_1.loc[:,"tweet"] = Data_train_1["tweet"].apply(lambda X_train: pre_Process_SS(X_train))

In [ ]:
Data_test_1

,tweet,labels
0,chines convert islam realis muslim affect #cor...,1
1,11 13 peopl ( diamond princess cruis ship ) in...,1
2,"covid - 19 caus bacterium , viru treat aspirin",1
3,mike penc rnc speech prais donald trump 's cov...,1
4,6/10 sky 's @user explain latest #covid19 data...,0
...,...,...
2135,donald trump wrongli claim new zealand “ big s...,1
2136,current understand #covid19 spread mostli pers...,0
2137,noth scream “ sat around fuck lockdown ” quit ...,1
2138,birx say covid - 19 outbreak control ‘ peopl m...,1


In [ ]:
Data_train_1

,tweet,labels
0,cdc current report 99031 death . gener discrep...,0
1,state report 1121 death small rise last tuesda...,0
2,polit correct woman ( almost ) use pandem excu...,1
3,#indiafightscorona : 1524 #covid test laborato...,0
4,popul state gener larg case count look new cas...,0
...,...,...
6415,tiger test posit covid - 19 pleas stay away pe...,1
6416,? ? ? autopsi prove covid - 19 ? ? � blood clo...,1
6417,_a post claim covid - 19 vaccin alreadi develo...,1
6418,aamir khan donat 250 cr . pm relief care fund,1


**Vecto hoa du lieu**

In [ ]:
X_train_1=Data_train_1.iloc[:,0].values
Y_train_1=Data_train_1.iloc[:,1].values

In [ ]:
X_test_1=Data_test_1.iloc[:,0].values
Y_test_1=Data_test_1.iloc[:,1].values

In [ ]:
TF_IDF_vectorizer = TfidfVectorizer()

In [ ]:
X_train_1 = TF_IDF_vectorizer.fit_transform(X_train_1).toarray()
X_test_1 = TF_IDF_vectorizer.transform(X_test_1).toarray()

**Train** + **Evalute** 

In [ ]:
def Evaluation_metrics(Y_test, Y_Pred):
  print('accuracy ',accuracy_score(Y_test,Y_Pred))
  print('precision ', precision_score(Y_test,Y_Pred,average= 'weighted'))
  print('recall ', recall_score(Y_test,Y_Pred,average= 'weighted'))
  print("f1", f1_score(Y_test,Y_Pred, average= 'weighted'))
  #print(classification_report(Y_test, Y_Pred, target_names = ["FAKE","REAL"]))
  #confusion_matrix(Y_test,Y_Pred, labels=[0,1])
  cm = confusion_matrix(Y_test,Y_Pred)
  print(cm)

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_1):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_1[train_index], X_train_1[test_index]
  y_train, y_test = Y_train_1[train_index], Y_train_1[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classsifier_NB =GaussianNB()
  classsifier_NB.fit(x_train, y_train)
  Y_pred = classsifier_NB.predict(x_test)
  Y_pred = Y_pred.reshape(-1,1)
  #cm = confusion_matrix(y_test,Y_pred_1)
  #print(cm)
  #print (Y_pred_1.shape)
  #print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8255451713395638
precision  0.8306762292743601
recall  0.8255451713395638
f1 0.8237163557961132
[[622  66]
 [158 438]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

accuracy  0.823208722741433
precision  0.8310886597576451
recall  0.823208722741433
f1 0.8219309339399683
[[587  63]
 [164 470]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8535825545171339
precision  0.856662927235086
recall  0.8535825545171339
f1 0.852824755216871
[[612  61]
 [127 484]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8115264797507789
precision  0.8187609286080376
recall  0.8115264797507789
f1 0.8098743561471956
[[591  70]
 [172 451]]


In [ ]:
y_pred = classsifier_NB.predict(X_test_1)

In [ ]:
Y_test_1=Y_test_1.reshape(-1,1)

In [ ]:
y_pred=y_pred.reshape(-1,1)

In [ ]:
print(Y_test_1.shape)
print(y_pred.shape)

(2140, 1)
(2140, 1)


In [ ]:
Evaluation_metrics(Y_test_1, y_pred)

accuracy  0.8317757009345794
precision  0.8348979917395565
recall  0.8317757009345794
f1 0.8308458354790645
[[998 122]
 [238 782]]


Logistic regression

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_1):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_1[train_index], X_train_1[test_index]
  y_train, y_test = Y_train_1[train_index], Y_train_1[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classifier_LR_Val =LogisticRegression()
  classifier_LR_Val.fit(x_train, y_train)
  Y_pred = classifier_LR_Val.predict(x_test)
  Y_pred = Y_pred.reshape(-1,1)
  cm = confusion_matrix(y_test,Y_pred)
  print(cm)
  print (Y_pred.shape)
  print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[622  66]
 [ 26 570]]
(1284, 1)
(1284, 1)
accuracy  0.9283489096573209
precision  0.9303317796920729
recall  0.9283489096573209
f1 0.9284394555954242
[[622  66]
 [ 26 570]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[[601  49]
 [ 46 588]]
(1284, 1)
(1284, 1)
accuracy  0.92601246105919
precision  0.9260260725181496
recall  0.92601246105919
f1 0.9260142114606302
[[601  49]
 [ 46 588]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[619  54]
 [ 42 569]]
(1284, 1)
(1284, 1)
accuracy  0.9252336448598131
precision  0.9254498810916326
recall  0.9252336448598131
f1 0.9252608961124793
[[619  54]
 [ 42 569]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[597  64]
 [ 41 582]]
(1284, 1)
(1284, 1)
accuracy  0.9182242990654206
precision  0.9188478047854873
recall  0.9182242990654206
f1 0.9182414138957183
[[597  64]
 [ 41 582]]


In [ ]:
y_pred_LR = classifier_LR_Val.predict(X_test_1)

In [ ]:
Evaluation_metrics(Y_test_1, y_pred_LR)

accuracy  0.9210280373831776
precision  0.9210218973166563
recall  0.9210280373831776
f1 0.9210150890371182
[[1039   81]
 [  88  932]]


SVM

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_1):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_1[train_index], X_train_1[test_index]
  y_train, y_test = Y_train_1[train_index], Y_train_1[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classifier_SVC = SVC(kernel ="rbf")  
  classifier_SVC.fit(x_train, y_train)
  Y_pred = classifier_SVC.predict(x_test)
  Y_pred = Y_pred.reshape(-1,1)
  #cm = confusion_matrix(y_test,Y_pred)
  #print(cm)
  #print (Y_pred.shape)
  #print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9431464174454829
precision  0.9446860349836608
recall  0.9431464174454829
f1 0.9432153504797999
[[634  54]
 [ 19 577]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

accuracy  0.9384735202492211
precision  0.9388013339386917
recall  0.9384735202492211
f1 0.9384728858228822
[[602  48]
 [ 31 603]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9447040498442367
precision  0.9448805225544161
recall  0.9447040498442367
f1 0.9447228955256005
[[632  41]
 [ 30 581]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9353582554517134
precision  0.9360929877152765
recall  0.9353582554517134
f1 0.9353709996025488
[[607  54]
 [ 29 594]]


In [ ]:
y_pred_SVC = classifier_SVC.predict(X_test_1)

In [ ]:
Evaluation_metrics(Y_test_1, y_pred_SVC)

accuracy  0.9294392523364486
precision  0.9295428260214874
recall  0.9294392523364486
f1 0.9294567072062007
[[1038   82]
 [  69  951]]
